<a href="https://colab.research.google.com/github/rockatoo/durable_rules_task/blob/main/%EC%82%B0%EC%97%85%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B0%9C%EB%A1%A0_5%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install durable_rules

In [2]:
from durable.lang import *

주문 클래스



In [3]:
class Product:
  def __init__(self, name, price):
    self.name = name
    self.price = price
  
  def get_name(self):
    return self.name
  
  def get_price(self):
    return self.price


class Products(Product):
  def __init__(self, product, ea):
    super().__init__(product.name, product.price)
    self.ea = ea
  
  def get_total_price(self):
    return self.price * self.ea

class ProductsList:
  def __init__(self, products_tuple):
    self.products_tuple = list(products_tuple) if isinstance(products_tuple, (tuple, list)) else list()
  
  def get_total_price(self):
    total_price = 0
    for products in self.products_tuple:
      total_price += products.get_total_price()
    
    return total_price

class Order:
  def __init__(self, products_list, payment_price):
    self.products_list = products_list
    self.payment_price = payment_price
  
  def get_products_price(self):
    return self.products_list.get_total_price()
  
  def get_payment_price(self):
    return self.payment_price

In [6]:
products_tuple = (
    Products(Product('웨하스', 3900), 4),
    Products(Product('단백질 보충제', 18900), 2),
    Products(Product('커피믹스 60개', 8000), 3)
)
products_list = ProductsList(products_tuple)
order = Order(products_list, 77400)

with ruleset('Order Validation'):
    @when_all(m.products_price == m.payment_price)
    def payment(c):
      print('payment is correct')

In [7]:
post('Order Validation', {
    'products_price': order.get_products_price(),
    'payment_price': order.get_payment_price()
    })

{'$s': 1,
 'exception': 'exception caught {\'payment\': \'correct\', \'sid\': \'0\'}, traceback [\'  File "/usr/local/lib/python3.7/dist-packages/durable/engine.py", line 241, in run\\n    self._func(c)\\n\', \'  File "<ipython-input-6-24129b3f41a6>", line 12, in payment\\n    c.assert_fact({\\\'payment\\\': \\\'correct\\\'})\\n\', \'  File "/usr/local/lib/python3.7/dist-packages/durable/engine.py", line 88, in assert_fact\\n    self._ruleset.assert_fact(fact)\\n\', \'  File "/usr/local/lib/python3.7/dist-packages/durable/engine.py", line 343, in assert_fact\\n    return self._handle_result(durable_rules_engine.assert_fact(self._handle, json.dumps(fact, ensure_ascii=False)), fact)\\n\', \'  File "/usr/local/lib/python3.7/dist-packages/durable/engine.py", line 328, in _handle_result\\n    raise MessageNotHandledException(message)\\n\']',
 'id': 'sid-0',
 'sid': '0'}